In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)


RNN are good for sequential data processing — NLP tasks and time-series predictions.

A typical RNN for NLP architecture is an embedding layer (pretrained or not) and a sequence of bidirectional LSTM layers since all text is visible to the model immediately.
RNN for time-series predictions is typically one-directional, although using bidirectional layers may improve the quality too. 1D convolutions and other techniques and tricks are also widely used here.

In [ ]:
# RNN for NLP
model = tf.keras.Sequential([
    layers.Embedding(vocab_size,
                     embedding_dim,
                     input_length=max_length),
    layers.Bidirectional(layers.LSTM(32, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(16)),
    layers.Dropout(0.3),
    layers.Dense(16, activation='relu'),
    layers.Dense(6, activation='softmax')  # multiclass classification
])

# RNN for time series
model = tf.keras.Sequential([
    layers.Conv1D(filters=128, kernel_size=7,
                  strides=1, padding="causal",
                  activation="relu",
                  input_shape=(WINDOW_SIZE, 1)),
    # univariate time series - predict a value based on
    # 'WINDOW_SIZE' previous steps for 1 feature

    layers.LSTM(32, return_sequences=True),
    layers.LSTM(16, return_sequences=True),
    layers.Dense(16, activation="relu"),
    layers.Dense(1)  # predict one value
])

# explore your model structure
model.summary()

More Complex Neural Network Architectures with Functional API

Functional API allows you to add non-sequential connections in your network, and specify multiple inputs and outputs. Below is a rather cumbersome, but demonstrative description of a network that takes text, an image, and several numbers as input features.

In [ ]:
# variable-size input for text
text_input = tf.keras.Input(shape=(None,), name="text_input")

# input for images IMG_HEIGHT x IMG_WEGTH x CHANNELS
image_input = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS),
                             name="image_input")

# input for numbers ('num_inputs' variables)
numbers_input = tf.keras.Input(shape=(num_inputs,), name="numbers_input")

# processing text
text_features = layers.Embedding(max_length,
                                 embedding_dim,
                                 input_length=max_length)(text_input)
text_features_buf = layers.Bidirectional(
    layers.LSTM(64, return_sequences=True))(text_features)
text_features = layers.Bidirectional(layers.LSTM(
    64, return_sequences=True))(text_features_buf)
text_features = layers.Bidirectional(
    layers.LSTM(64, return_sequences=True))(text_features)
text_features = layers.Add()([text_features, text_features_buf])
text_features = layers.LayerNormalization()(text_features)
text_features = layers.Bidirectional(layers.LSTM(32))(text_features)

# processing images
image_features = layers.Conv2D(32, (3, 3), activation='relu')(image_input)
image_features = layers.MaxPooling2D(2, 2)(image_features)
image_features = layers.Conv2D(32, (3, 3), activation='relu')(image_features)
image_features = layers.MaxPooling2D(2, 2)(image_features)
image_features = layers.Flatten()(image_features)

# processing numbers
numbers_features = layers.Dense(units=64, activation='relu')(numbers_input)
numbers_features_buf = layers.Dense(
    units=32, activation='relu')(numbers_features)
numbers_features = layers.Dense(
    units=32, activation='relu')(numbers_features_buf)
numbers_features = layers.Dense(units=32, activation='relu')(numbers_features)
numbers_features = layers.Add()([numbers_features, numbers_features_buf])
numbers_features = layers.LayerNormalization()(numbers_features)

# merge all available features into a single large vector
concat = layers.concatenate([text_features,
                             image_features,
                             numbers_features])

intermediate_output = tf.keras.layers.Dense(64, activation='relu')(concat)

# final part of the model
dense_1 = layers.Dense(64, activation='relu')(concat)
dropout_1 = layers.Dropout(0.2)(dense_1)
dense_2 = layers.Dense(32, activation='relu')(dropout_1)
dropout_2 = layers.Dropout(0.2)(dense_2)
output = layers.Dense(1, activation='relu')(dropout_2)

model = tf.keras.Model(inputs=[text_input, image_input, numbers_input],
                       outputs=[intermediate_output, output])


from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True,
           show_layer_names=False, to_file='data/img/model.png')

Preparing Data for Computer Vision and NLP tasks

Using ImageDataGenerator
ImageDataGenerator will help you a lot in computer vision tasks —it will label your images automatically based on directory structure or perform in-memory data augmentation


In [ ]:
# augmented ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# default ImageDataGenerator without augmentation
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'  # 'categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'  # 'categorical'
)

# Another option is not to specify the validation set explicitly,
# but to entrust the split to ImageDataGenerator
# In this case, be sure that the seed parameter is the same
# for both sets, otherwise the sets may overlap

train_datagen = ImageDataGenerator(rescale=1./255,
                                   # set validation split ratio
                                   validation_split=0.2)  

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',  # 'categorical'
    subset='training',  # set as training data
    seed=SEED)

validation_generator = train_datagen.flow_from_directory(
    train_dir,  # the same directory
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',  # 'categorical'
    subset='validation',  # set as validation data
    seed=SEED)

Tokenizing and padding sentences for NLP tasks

Tokenization and padding sentences are common practices for NLP tasks. First, you turn sentences into vectors, and then you make sure that all these vectors have a fixed length to feed them to the input of your model.

![title]("RNN Tokenizer anbd Pd Seq.png")


In [ ]:
# default parameters
vocab_size = 1000
embedding_dim = 32
max_length = 100
trunc_type = "post"
padding_type = "post"
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences,
                                maxlen=max_length,
                                padding=padding_type,
                                truncating=trunc_type)

validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences,
                                  maxlen=max_length,
                                  padding=padding_type)